In [1]:
import ants
import glob, os

from nilearn import plotting

In [54]:
# 
bids_dir2 = '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2'
bids_dir3 = '/home/lauri/Documents/TMS-FDG/derivatives/iter6s3'


fullpath_subjects = glob.glob(f'{bids_dir}/sub-*/')
print(fullpath_subjects)
subjects = []
for s in fullpath_subjects:
    subjects.append(os.path.basename(os.path.normpath(s)))



['/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg001/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg005/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg004/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg007/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg011/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg019/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg031/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg010/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg006/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg015/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg002/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg008/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg034/', '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg009/', '/home/lauri/Documents/TMS-FDG/derivatives/iter

In [55]:
def warp2MNI(bids_dir, subject, session):
    
    SUVR_pet_file = f'{bids_dir}/{subject}/{session}/pet/SUVR.nii.gz'
    T1w_file = glob.glob(f'{bids_dir}/{subject}/{session}/anat/{subject}_{session}_desc-preproc_T1w.nii.gz')[0]
    T1w_MNI_file = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*_res-2_desc-preproc_T1w.nii.gz')[0]

    T1w2MNI = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*MNI152NLin6Asym_mode-image_xfm.h5')[0]
    REGDIR=f'{bids_dir}/{subject}/{session}/pet/antsr/'

    print(f'processing {subject} {session}')
    # see if files exist:
    if not os.path.exists(SUVR_pet_file):
        print(f'{subject} PET data missing')
        return
    
    if not os.path.exists(T1w_MNI_file):
        print(f'{subject} T1w in MNI missing')
        return
    
    if not os.path.exists(T1w_file) :
        print(f'{subject} T1w missing')
        return
    
    if not os.path.exists(T1w2MNI):
        print(f'{subject} T1w2MNI missing')
        return
    
    # regdir 
    if not os.path.exists(REGDIR ):
        os.makedirs(REGDIR)
        
    # load as ants images 
    SUVR_pet = ants.image_read(SUVR_pet_file)
    T1w = ants.image_read(T1w_file)
    T1w_MNI = ants.image_read(T1w_MNI_file)

    # calculate PET2T1w
    registration = ants.registration(fixed=T1w, moving=SUVR_pet, type_of_transform='Affine',metric='MI', outprefix=REGDIR)

    img = registration['warpedmovout']

    T1w_mask_file = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*_desc-brain_mask.nii.gz')[0]
    T1w_mask  = ants.image_read(T1w_mask_file)
    
    # combine registrations and save combined transform file
    comptx_file = ants.apply_transforms(fixed=T1w_MNI,
                                        moving=SUVR_pet,
                                        transformlist=[ T1w2MNI, registration['fwdtransforms'][0]],
                                        compose=REGDIR)

    # apply PET to MNI transform to dynamic_template
    PET_MNIspace = ants.apply_transforms(fixed=T1w_MNI,moving=SUVR_pet,transformlist=[comptx_file])
    #ants.image_write(template_MNIspace,os.path.join(REGDIR,'dynamic_template_MNI.nii.gz'))
    ants.image_write(PET_MNIspace,f'{bids_dir}/{subject}/{session}/pet/SUVR_in_MNI.nii.gz')
    #disp = plotting.plot_stat_map(f'{bids_dir}/{subject}/{session}/pet/SUVR_in_MNI.nii.gz', alpha= 2/4)
    #disp.savefig(f'{bids_dir}/{subject}/{session}/pet/SUVR_in_MNI.nii.png')
    print(f'{subject} {session} done')
    

In [58]:
sessions = ['ses-002', 'ses-003']

for subject in subjects:
    warp2MNI(bids_dir2, subject, 'ses-002')
    warp2MNI(bids_dir3, subject, 'ses-003')


processing sub-tbsfdg001 ses-002
sub-tbsfdg001 ses-002 done
processing sub-tbsfdg001 ses-003
sub-tbsfdg001 PET data missing
processing sub-tbsfdg005 ses-002
sub-tbsfdg005 PET data missing
processing sub-tbsfdg005 ses-003
sub-tbsfdg005 PET data missing
processing sub-tbsfdg004 ses-002
sub-tbsfdg004 PET data missing
processing sub-tbsfdg004 ses-003
sub-tbsfdg004 PET data missing
processing sub-tbsfdg007 ses-002
sub-tbsfdg007 PET data missing
processing sub-tbsfdg007 ses-003
sub-tbsfdg007 PET data missing
processing sub-tbsfdg011 ses-002
sub-tbsfdg011 PET data missing
processing sub-tbsfdg011 ses-003
sub-tbsfdg011 PET data missing
processing sub-tbsfdg019 ses-002
sub-tbsfdg019 PET data missing
processing sub-tbsfdg019 ses-003
sub-tbsfdg019 PET data missing
processing sub-tbsfdg031 ses-002
sub-tbsfdg031 PET data missing
processing sub-tbsfdg031 ses-003
sub-tbsfdg031 PET data missing
processing sub-tbsfdg010 ses-002
sub-tbsfdg010 PET data missing
processing sub-tbsfdg010 ses-003
sub-tbsfdg0

IndexError: list index out of range